In [140]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import StackingClassifier
from sklearn.impute import SimpleImputer    
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from xgboost import XGBRegressor    
from sklearn.model_selection import GridSearchCV, StratifiedKFold, RandomizedSearchCV, KFold    
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imb_pipeline
from sklearn.pipeline import Pipeline
import lightgbm as lgb
from catboost import CatBoostRegressor
from IPython.display import clear_output
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel, RFE
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer

In [141]:
data = pd.read_csv('data/train.csv')
original = pd.read_csv('archive/CrabAgePrediction.csv')

In [142]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74051 entries, 0 to 74050
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              74051 non-null  int64  
 1   Sex             74051 non-null  object 
 2   Length          74051 non-null  float64
 3   Diameter        74051 non-null  float64
 4   Height          74051 non-null  float64
 5   Weight          74051 non-null  float64
 6   Shucked Weight  74051 non-null  float64
 7   Viscera Weight  74051 non-null  float64
 8   Shell Weight    74051 non-null  float64
 9   Age             74051 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 5.6+ MB


In [143]:
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3893 entries, 0 to 3892
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             3893 non-null   object 
 1   Length          3893 non-null   float64
 2   Diameter        3893 non-null   float64
 3   Height          3893 non-null   float64
 4   Weight          3893 non-null   float64
 5   Shucked Weight  3893 non-null   float64
 6   Viscera Weight  3893 non-null   float64
 7   Shell Weight    3893 non-null   float64
 8   Age             3893 non-null   int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 273.9+ KB


In [144]:
new_data = pd.concat([data.drop('id', axis=1), original])
new_data.shape

(77944, 9)

In [145]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77944 entries, 0 to 3892
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sex             77944 non-null  object 
 1   Length          77944 non-null  float64
 2   Diameter        77944 non-null  float64
 3   Height          77944 non-null  float64
 4   Weight          77944 non-null  float64
 5   Shucked Weight  77944 non-null  float64
 6   Viscera Weight  77944 non-null  float64
 7   Shell Weight    77944 non-null  float64
 8   Age             77944 non-null  int64  
dtypes: float64(7), int64(1), object(1)
memory usage: 5.9+ MB


In [146]:
epsilon = 1e-7  # a small constant

def engineer_features(data):
    data['shell_to_total_weight_ratio'] = data['Shell Weight'] / (data['Weight'] + epsilon)
    data['shucked_to_total_weight_ratio'] = data['Shucked Weight'] / (data['Weight'] + epsilon)    
    
    # Adding interaction terms
    data['interaction_1'] = data['Shell Weight'] * data['Weight']
    data['interaction_2'] = data['Shucked Weight'] * data['Weight']
    
    return data


In [147]:
# Engineer features
new_data = engineer_features(new_data)

In [148]:
new_data = new_data.sample(100)

In [149]:
for col in new_data.columns:
    if new_data[col].dtype == np.float64:
        new_data[col] = new_data[col].astype(np.float32)

In [150]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 22878 to 5289
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Sex                            100 non-null    object 
 1   Length                         100 non-null    float32
 2   Diameter                       100 non-null    float32
 3   Height                         100 non-null    float32
 4   Weight                         100 non-null    float32
 5   Shucked Weight                 100 non-null    float32
 6   Viscera Weight                 100 non-null    float32
 7   Shell Weight                   100 non-null    float32
 8   Age                            100 non-null    int64  
 9   shell_to_total_weight_ratio    100 non-null    float32
 10  shucked_to_total_weight_ratio  100 non-null    float32
 11  interaction_1                  100 non-null    float32
 12  interaction_2                  100 non-null  

EDA

In [151]:
X = new_data.drop(columns=['Age'])
y = new_data['Age']

In [152]:
# Get numerical and categorical columns
num_columns = list(X.select_dtypes(exclude='object').columns)
cat_columns = list(X.select_dtypes(include='object').columns)

In [153]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [154]:
preprocessor = ColumnTransformer(
                    transformers=[('num', PowerTransformer(method='yeo-johnson'), num_columns),
                                  ('cat', OneHotEncoder(handle_unknown="ignore"), cat_columns)] 
                                ) 

In [155]:
X_train.columns

Index(['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'shell_to_total_weight_ratio',
       'shucked_to_total_weight_ratio', 'interaction_1', 'interaction_2'],
      dtype='object')

In [156]:
# base_models = [
#         ('xgb', XGBRegressor(random_state=42, tree_method='gpu_hist')),
#         ('rf', RandomForestRegressor(random_state=42)),
#         ('lgb', lgb.LGBMRegressor(random_state=42, device='gpu')),
#         ('cat', CatBoostRegressor(random_state=42, silent=True, task_type='GPU'))
#     ]


# feature_importances = {}

# for model_name, model in base_models:
#     pipeline = Pipeline(steps=[ 
#                                 ('Preprocessor', preprocessor),
#                                 ('Scaler', StandardScaler()),
#                                 ('Regressor', model)
#                               ])
#     pipeline.fit(X_train, y_train)
#     if hasattr(model, "feature_importances_"):
#         feature_importances[model_name] = model.feature_importances_
#     elif hasattr(model, "coef_"):
#         feature_importances[model_name] = model.coef_


In [157]:
# # Extract transformed feature names
# cat_feature_names = pipeline.named_steps['Preprocessor'].named_transformers_['cat'].get_feature_names_out(input_features=cat_columns)

# # Combine numerical and categorical feature names
# # Note: cat_feature_names comes first to align with how the model sees the features
# feature_names = cat_feature_names.tolist() + num_columns

In [158]:
# # Assuming your feature names are available in feature_names variable
# importance_df = pd.DataFrame(feature_importances, index=feature_names)

# # Take mean of feature importances across models
# importance_df['mean_importance'] = importance_df.mean(axis=1)

# # Select top n features
# n = 10  # or whatever number of features you want to keep
# top_features = importance_df.nlargest(n, 'mean_importance').index

# # # Filter your data to only include these top features

In [159]:
# top_features = top_features.to_list()
# if 'Sex_F' in top_features:
#     top_features.remove('Sex_F')
# if 'Sex_I' in top_features:
#     top_features.remove('Sex_I')
# if 'Sex_M' in top_features:
#     top_features.remove('Sex_M')    
# if 'Sex' not in top_features:
#     top_features.append('Sex')

In [160]:
# X_train = X_train[top_features]
# X_test = X_test[top_features]

In [161]:
# top_features

In [ ]:
%%time

# Define the base models
base_models = [
    ('xgb', XGBRegressor(random_state=42, tree_method='gpu_hist')),
#     ('rf', RandomForestRegressor(random_state=42)),
    ('lgb', lgb.LGBMRegressor(random_state=42, device='gpu')),
    ('cat', CatBoostRegressor(random_state=42, silent=True, task_type='GPU'))
]

#     # Define meta learner model
final_model = LinearRegression()

# Define the stacking ensemble
stacking_regressor = StackingRegressor(estimators=base_models, final_estimator=final_model, cv=5)


feature_engineering = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)


# Define the feature selection method
feature_selection = RFE(RandomForestRegressor(n_estimators=100), n_features_to_select=7)

# Preparing the pipeline architecture
pipeline = Pipeline(steps=[ 
                            ('Preprocessor', preprocessor),
                            ('pca', PCA(n_components = 10)),
#                             ('Scaler', StandardScaler()),
                            ('Regressor', stacking_regressor)
                          ]
                       )

# cross validation
kFold = KFold(n_splits=5)


# Define the hyperparameters for each model in the ensemble
params = { 
'Regressor__xgb__n_estimators': [500, 1000],
'Regressor__xgb__max_depth': [15],
'Regressor__xgb__learning_rate': [0.01],
'Regressor__xgb__subsample': [0.8],
'Regressor__xgb__tree_method': ['gpu_hist'],
# 'Regressor__rf__n_estimators': [ 600, 1000],
# 'Regressor__rf__max_depth': [20],
# 'Regressor__rf__min_samples_split': [5],
# 'Regressor__rf__min_samples_leaf': [5],
'Regressor__lgb__n_estimators': [300, 600, 900],
'Regressor__lgb__max_depth': [6],
'Regressor__lgb__learning_rate': [0.01],
'Regressor__lgb__subsample': [0.6],
'Regressor__lgb__device': ['gpu'],
'Regressor__cat__iterations': [500, 1000],
'Regressor__cat__depth': [6],
'Regressor__cat__learning_rate': [0.05],
'Regressor__cat__task_type': ['GPU']
}

# Update hyperparameters range
params_xgb = {
               'Regressor__max_depth': [6, 8, 10, 12],
               'Regressor__learning_rate': [0.01, 0.03, 0.05, 0.1],
               'Regressor__subsample': [0.6, 0.7, 0.8, 0.9],
               'Regressor__colsample_bytree': [0.4, 0.5, 0.6, 0.7, 0.8, 1.0],
               'Regressor__n_estimators': [100, 200, 500, 1000],
               'Regressor__tree_method': ['gpu_hist'],
             }
# Initialize hyper parameter tuning using RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=pipeline,
                                   param_distributions=params,
                                   scoring='neg_mean_absolute_error',
                                   n_iter=50,  # increase this value as needed
                                   cv=kFold,
                                   n_jobs=-1,
                                   verbose=3,
                                   random_state=42)
random_search.fit(X_train, y_train)

best_params = random_search.best_params_

cv_score = random_search.best_score_
test_score = random_search.score(X_test, y_test) 

print(f'Cross-validation score: {cv_score}\nTest score: {test_score}')

C:\Users\RAHUL DATTA\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:305: UserWarning: The total space of parameters 12 is smaller than n_iter=50. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

# Define a function to plot learning curves
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")
    plt.legend(loc="best")
    return plt

# Generate the plot
plot_learning_curve(random_search.best_estimator_, "Learning curve", X, y, cv=KFold(n_splits=5))

In [ ]:
# grid_search.best_params_
random_search.best_params_
# grid_search.best_params_

Error Analysis

In [ ]:
y_pred = random_search.predict(X_test)
# y_pred = grid_search.predict(X_test)
errors = abs(y_test - y_pred)

This can give you an idea about the distribution of errors (are they normal, skewed, etc.).

In [ ]:
import matplotlib.pyplot as plt

plt.hist(errors, bins=30)
plt.xlabel('Prediction Error')
plt.title('Histogram of prediction errors')
plt.show()


Identifying instances with high error:

In [ ]:
error_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred, 'Error': errors})
error_df.sort_values('Error', ascending=False, inplace=True)

Analyzing residuals vs. predicted plot.
    - A residuals vs. predicted plot can provide insight into whether my model 
      is systematically over or underpredicting at certain levels of the dependent variable.

In [ ]:
plt.scatter(y_pred, y_test - y_pred)
plt.xlabel('Predicted Age')
plt.ylabel('Residuals')
plt.axhline(y=0, color='r', linestyle='-')
plt.title('Residuals vs Predicted Age')
plt.show()

Errors across different groups

In [ ]:
error_df = X_test.copy()
error_df['Error'] = errors
error_df.groupby('Sex')['Error'].mean().plot(kind='bar')
plt.ylabel('Mean Absolute Error')
plt.title('Error across different genders')
plt.show()

Generating Submission file

In [ ]:
test = pd.read_csv('data/test.csv')
new_test = engineer_features(test)

ids = test.id
# y_pred = grid_search.predict(test)
y_pred = random_search.predict(new_test)
y_pred = np.round(y_pred)

In [ ]:
submission = pd.DataFrame({'id':ids, 'Age': y_pred})


In [ ]:
submission

In [ ]:
submission.to_csv('submission11.csv', index=False)

Neural Network

In [108]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


# Prepare the transformers for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), [1, 2, 3, 4, 5, 6]),
        ('cat', OneHotEncoder(), [0]),
    ]
)

# Apply transformations to X
X = preprocessor.fit_transform(X)

# Split the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the model
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

model.add(Dense(64, activation='relu'))

model.add(Dense(32, activation='relu'))

model.add(Dense(1))

# Compile the model
model.compile(optimizer=Adam(), loss='mean_absolute_error')

# Fit the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

# Evaluate the model
test_loss = model.evaluate(X_test, y_test)

print('Test loss:', test_loss)


Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 2.4271 - val_loss: 1.5332
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 1.6458 - val_loss: 1.5468
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 1.6008 - val_loss: 1.5583
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 1.5807 - val_loss: 1.4805
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 1.5564 - val_loss: 1.4379
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 1.5481 - val_loss: 1.4503
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 1.5123 - val_loss: 1.4402
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4967 - val_loss: 1.4194
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 1.5166 - val_loss: 1.4002
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 1.5066 - val_lo

250/250 [==============================] - 0s 1ms/step - loss: 1.4291 - val_loss: 1.3558
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4299 - val_loss: 1.3671
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4281 - val_loss: 1.3865
Epoch 84/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4333 - val_loss: 1.4568
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4331 - val_loss: 1.3715
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4292 - val_loss: 1.4074
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4344 - val_loss: 1.3997
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4320 - val_loss: 1.3681
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4285 - val_loss: 1.3473
Epoch 90/100
250/250 [==============================] - 0s 1ms/step - loss: 1.4218 - val_loss: 

In [109]:
test = pd.read_csv('data/test.csv')
X = engineer_features(test)
X = preprocessor.transform(X)

ids = test.id
y_pred = model.predict(X)
# y_pred = random_search.predict(new_test)
y_pred = np.round(y_pred)

1543/1543 [==============================] - 1s 636us/step


In [110]:
submission = pd.DataFrame({'id':ids, 'Age': y_pred.flatten()})
submission.to_csv('submission10_nn.csv', index=False)

In [111]:
submission

,id,Age
0,74051,7.0
1,74052,8.0
2,74053,10.0
3,74054,9.0
4,74055,7.0
...,...,...
49363,123414,10.0
49364,123415,7.0
49365,123416,14.0
49366,123417,10.0
